In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re
import csv
import glob as gl
import tqdm
import pandas as pd
import numpy as np

In [2]:
def prevailing_wind_direction(wind_dir_series):
    rad = np.deg2rad(wind_dir_series)
    sin_mean = np.nanmean(np.sin(rad))
    cos_mean = np.nanmean(np.cos(rad))
    mean_dir_rad = np.arctan2(sin_mean, cos_mean)
    mean_dir_deg = (np.rad2deg(mean_dir_rad)) % 360
    return mean_dir_deg

In [3]:
def download_file(file_url, download_dir):
    """Download the file from the specified URL."""
    local_filename = os.path.join(download_dir, os.path.basename(file_url))
    with requests.get(file_url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print(f"Downloaded: {local_filename}")
    return os.path.basename(file_url)

def scrape_and_download(url, file_pattern, download_dir):
    """Scrape the website and download files matching the pattern."""
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links on the webpage and download the matching files
    for link in soup.find_all('a', href=True):
        file_url = link['href']
        if file_pattern.match(file_url):
            # Prepend the base URL if necessary
            if not file_url.startswith('http'):
                file_url = url + file_url
            # Download the file
            download_file(file_url, download_dir)
            
def download_vito(year):
    # Directory where you want to save the downloaded files
    download_dir = '../../Data/vito'
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    
    # Pattern to match the file names
    file_pattern = re.compile(r'vto.*q1h\.txt')

    # Find data online
    url = 'https://amrc.ssec.wisc.edu/data/ftp/pub/aws/q1h/' + str(year) + '/'

    scrape_and_download(url, file_pattern, download_dir)

In [4]:
for year in [2004, 2005, 2006, 2007, 2008, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]:
    download_vito(year)

Downloaded: ../../Data/vito\vto200402q1h.txt
Downloaded: ../../Data/vito\vto200403q1h.txt
Downloaded: ../../Data/vito\vto200404q1h.txt
Downloaded: ../../Data/vito\vto200405q1h.txt
Downloaded: ../../Data/vito\vto200406q1h.txt
Downloaded: ../../Data/vito\vto200407q1h.txt
Downloaded: ../../Data/vito\vto200408q1h.txt
Downloaded: ../../Data/vito\vto200409q1h.txt
Downloaded: ../../Data/vito\vto200410q1h.txt
Downloaded: ../../Data/vito\vto200411q1h.txt
Downloaded: ../../Data/vito\vto200412q1h.txt
Downloaded: ../../Data/vito\vto200501q1h.txt
Downloaded: ../../Data/vito\vto200502q1h.txt
Downloaded: ../../Data/vito\vto200503q1h.txt
Downloaded: ../../Data/vito\vto200504q1h.txt
Downloaded: ../../Data/vito\vto200505q1h.txt
Downloaded: ../../Data/vito\vto200506q1h.txt
Downloaded: ../../Data/vito\vto200507q1h.txt
Downloaded: ../../Data/vito\vto200508q1h.txt
Downloaded: ../../Data/vito\vto200509q1h.txt
Downloaded: ../../Data/vito\vto200510q1h.txt
Downloaded: ../../Data/vito\vto200511q1h.txt
Downloaded

In [5]:
path = '../../Data/vito/'
# create list of file names
aws_files = gl.glob(path + "/vto*.txt")
# create data frame
columns = ['year', 'doy', 'month', 'day', 'hour', 'temperature', 'pressure', 'wind_speed', 'wind_direction', 'relative_humidity', '?']
content = []
for filename in tqdm.tqdm(aws_files, total=len(aws_files)):
    df = pd.read_csv(filename, names=columns, sep='\s+')
    df.drop(index=df.index[:2], axis=0, inplace=True)
    content.append(df)
vito = pd.concat(content)
# create datetime column and set as index:
vito['DateTime'] = vito['year'].astype(str) + '-' + vito['month'].astype(str) + '-' + vito['day'].astype(str) + ' ' + vito['hour'].astype(str)
vito['DateTime'] = pd.to_datetime(vito['DateTime'],format='%Y-%m-%d %H%M')
vito.index = vito['DateTime']
del vito['DateTime']
# make data into floats
for i in vito.columns:
    vito[i] = vito[i].astype(float)
# replace fill values with nans:
vito.replace(444.0, np.nan, inplace=True)
# resample to daily resolution
del vito['hour']
agg_dict = dict.fromkeys([col for col in vito.columns if col != 'wind_direction'], 'mean')
agg_dict['wind_direction'] = prevailing_wind_direction
vito = vito.resample('1D').agg(agg_dict)

vito

100%|████████████████████████████████████████████████████████████████████████████████| 243/243 [00:06<00:00, 36.35it/s]
C:\Users\Homo Naledi\AppData\Local\Temp\ipykernel_11416\2800808839.py:3: RuntimeWarning: Mean of empty slice
  sin_mean = np.nanmean(np.sin(rad))
C:\Users\Homo Naledi\AppData\Local\Temp\ipykernel_11416\2800808839.py:4: RuntimeWarning: Mean of empty slice
  cos_mean = np.nanmean(np.cos(rad))


,year,doy,month,day,temperature,pressure,wind_speed,relative_humidity,?,wind_direction
DateTime,,,,,,,,,,
2004-02-01,2004.0,32.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2004-02-02,2004.0,33.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2004-02-03,2004.0,34.0,2.0,3.0,-14.666667,983.766667,NaN,59.966667,NaN,NaN
2004-02-04,2004.0,35.0,2.0,4.0,-17.873913,986.339130,NaN,69.478261,NaN,NaN
2004-02-05,2004.0,36.0,2.0,5.0,-15.680952,995.486957,NaN,63.400000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-12-27,2024.0,362.0,12.0,27.0,-6.210526,987.683333,2.616667,72.916667,NaN,55.812919
2024-12-28,2024.0,363.0,12.0,28.0,-5.866667,987.887500,4.216667,78.837500,NaN,176.100893
2024-12-29,2024.0,364.0,12.0,29.0,-5.005882,990.737500,1.691667,67.220833,NaN,190.416911


In [6]:
vito.to_csv('../../Data/vito/vito.csv')